In [18]:
import numpy as np
import pickle

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
model=load_model('next_word_lstm.keras')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 13, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,658,256 (13.96 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,438,838 (9.30 MB)

In [20]:
with open('tokenizer.picke','rb') as handle:
    tokenizer=pickle.load(handle)
print('tokenizer downloaded succefully')

tokenizer downloaded succefully


In [21]:
max_seq_len=14

In [30]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    token_list=token_list[-(max_sequence_len-1):]
    
    token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted_probs=model.predict(token_list,verbose=0)
    predicted_index=np.argmax(predicted_probs[0])
    return tokenizer.index_word.get(predicted_index,None)

In [32]:
input_text='to be bad is better than'
print('input text:',input_text)

next_word=predict_next_word(
    model,tokenizer,input_text,max_seq_len
)
print(next_word)

input text: to be bad is better than
or


In [36]:
def generate_text(model, tokenizer, seed_text, max_sequence_len, no_next_words=10):
    
    output_text = seed_text
    
    for _ in range(no_next_words):
        next_word = predict_next_word(
            model,
            tokenizer,
            output_text,
            max_sequence_len
        )
        
        if next_word is None:
            break
            
        output_text += " " + next_word
    
    return output_text

In [35]:
seed_text = "to be or not to be"

'''
sample input:

EXACT lines / phrases from Hamlet:
“to be or not”
“alas poor yorick”
“frailty thy name”
“something is rotten”
“o my prophetic soul”
“give me that man”

Hamlet-STYLE but NOT exact quotes
“the king is”
“my lord”
“i will speak”
“what is the matter”
“this is the”
“the queen my mother”
“i do not know”
“there is something”
“as i have said”
“the very instant”
'''

generated_text = generate_text(
    model,
    tokenizer,
    seed_text,
    max_seq_len,
    next_words=15
)

print(generated_text)

to be or not to be honest as heauen and earth so now ere wings take not downe thy selfe in
